In [1]:
import os
import re
import json
import time

import tiktoken
import pandas as pd
import numpy as np
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain
# from langchain.llms import HuggingFaceHub
from transformers import AutoTokenizer
from dotenv import load_dotenv

from textutils import extract_ref_from_text, get_number2drawing_dict, convert_refs_to_drawing_num

pd.set_option("display.max_columns", 999)
os.chdir('/Users/hayley/Documents/p4ds/patent_search')
load_dotenv() 


True

In [ ]:
# main.py 돌려서 나온 결과 다시 저장
# sample_data = pd.read_csv('data_preprocess/data.csv')
# sample_data.to_excel("data_preprocess/data.xlsx")

# 근데 코드에서 코너 케이스 때문에 처리가 안된 pdf 가 있어서 그거는 excel 파일 다운받아서 눈으로 보고 채움 -> 그러다보니까 \n이 안지워진 경우가 있어서 지웠음
# sample_data = pd.read_excel('data_preprocess/data.xlsx')
# for col in sample_data.columns:
#     sample_data[col] = sample_data[col].str.replace('\n', '')
# sample_data.to_excel("data_preprocess/data.xlsx")

In [ ]:
# ## negative sample data 추가
# sample_data = pd.read_excel("data_preprocess/sample_data_old.xlsx", index_col=0)
# negative_samples = pd.read_csv('pdf_process/data.csv')
# negative_samples['id'] = negative_samples['id'].astype(str)
# negative_samples = negative_samples.loc[negative_samples['id']!= '1020160014413 (1)']
    
# negative_samples = negative_samples.loc[~negative_samples['id'].isin(sample_data['id'])]
# sample_data = pd.concat([sample_data, negative_samples], axis=0)

# # add labels column
# sample_data['labels'] = ""
# sample_data.loc[sample_data['id']=='1020180014052', 'labels'] = 'source'
# sample_data.loc[sample_data['id'].isin(['1020050097605','1020177009557', '1020120156759']), 'labels'] = 'target'
# sample_data['labels'] = sample_data['labels'].fillna("negative")
# sample_data.to_excel("data_preprocess/sample_data.xlsx")

In [2]:
sample_data = pd.read_excel("data_preprocess/sample_data.xlsx")

# Baseline: text only embedding search

- Result: using 3 golden targets and 96 negative samples and 1 source, within top 4 retrieved chunks, all of them are from golden related patents.
- source: 1020180014052 (10-1990984 patent)
- target: 
    - KR1020070041937 A* (1020050097605 app)
    - KR1020170071490 A* (10-2017-7009557 app)
    - KR101414532 B1 (10-2012-0156759 app)
- negative:
    - all others from 100 crawled patents

In [ ]:
# c.f. open ai tokenizer 기준 token 수 세는 코드

# def num_tokens_from_string(string: str, encoding_name: str) -> int:
#     """Returns the number of tokens in a text string."""
#     encoding = tiktoken.get_encoding(encoding_name)
#     num_tokens = len(encoding.encode(string))
#     return num_tokens

# num_tokens_from_string("tiktoken is great!", "cl100k_base")

In [5]:
# # -------------------Open ai ------------------ # # # 잘됨
# TokenTextSplitter -> split by number of tokens. use tokenizer from OpenAI
# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
# # cl100k_base is the tokenizer for the latest embedding (v2)
# text_splitter = TokenTextSplitter.from_tiktoken_encoder("cl100k_base",
#                                                         chunk_size=500, 
#                                                         chunk_overlap=10)

# # -------------------Instructor -large ------------------- # # # 잘안됨
# tokenizer = AutoTokenizer.from_pretrained("hkunlp/instructor-large", use_fast=True)
# text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=300, chunk_overlap=10)
# # ---------------- Multilingual e5 ----------------- # # # 잘됨 
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large", use_fast=True)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=300, chunk_overlap=10)
# # ------------------- Kobert ----------------- # # # 내가 쓰는 방법을 잘 몰라서 그런지 에러가 남 ㅠㅠ;;
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=True)
# text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=300, chunk_overlap=10)


In [6]:
patent_chunk_dicts=[]
text_columns = ['요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
       '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명']

for i, row in sample_data.iterrows():
    patent_dict = dict( 
        # 특허 번호 따기
        application_number = str(row.id), # 출원 번호
        publication_number = '', # 공개 번호
        patent_number = '', # 등록 번호
        chunks = [],
    )
    print(f"patent {row.id} 's chunk sizes")
    
    # text column들을 돌면서, chunking 하기
    chunks = []
    drawing_nums_list = []
    refs_list = []
    for col in text_columns:
        if (col in ['도면의간단한설명', '부호의설명']) or (str(row[col]) == 'nan'): # 도면의 간단한 설명, 부호의 설명은 embed 하지 않음!
            continue
        curr_section_chunks = text_splitter.create_documents([str(row[col])], [{"application_number": str(row.id)}])
        chunks.extend(curr_section_chunks)
        
        print(col, len(curr_section_chunks), end=' | ')
        
        # # chunk 별로 reference 발생한 부호 찾기
        # for chnk in curr_section_chunks:
        #     refs = extract_ref_from_text(chnk)
        #     refs_list.append(refs)
            
        #     #  부호가 있었으면 drawing number 로 한번 또 치환하기
        #     if len(refs) > 0: # if not empty, convert found references to drawing numbers
        #         drawing_nums = convert_refs_to_drawing_num(refs, num2drawing_dicts[str(row.id)]['num2drawing'])
        #     else:
        #         drawing_nums = []
        #     drawing_nums_list.append(drawing_nums)
                
        
    # patent_dict['chunks'] = list(zip(chunks, zip(refs_list, drawing_nums_list)))
    patent_dict['chunks'] = chunks
    print()
    patent_chunk_dicts.append(patent_dict)    
    

Token indices sequence length is longer than the specified maximum sequence length for this model (1266 > 512). Running this sequence through the model will result in indexing errors


patent 1020180014052 's chunk sizes
요약 1 | 청구범위 1 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020120156759 's chunk sizes
요약 1 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020177009557 's chunk sizes
요약 1 | 청구범위 1 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020050097605 's chunk sizes
요약 1 | 청구범위 1 | 배경기술 1 | 해결하려는과제 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020080110005 's chunk sizes
요약 1 | 기술분야 1 | 배경기술 1 | 
patent 1020107017567 's chunk sizes
요약 1 | 청구범위 1 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020110134350 's chunk sizes
요약 1 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020120022594 's chunk sizes
요약 1 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020127022798 's chunk sizes
요약 1 | 청구범위 1 | 기술분야 1 | 배경기술 1 | 발명을실시하기위한구체적인내용 1 | 
patent 1020130013714

In [ ]:
# json.dump(patent_chunk_dicts, open('data_preprocess/sample_chunk_data.json', 'w'), ensure_ascii = False )

In [7]:
all_chunks = [x['chunks'] for x in patent_chunk_dicts if x['application_number'] != '1020180014052']
all_chunks = sum(all_chunks, [])
# print(len(all_chunks))
# # -------------------Open ai ------------------ # # # 잘됨
# embeddings = OpenAIEmbeddings()

# # -------------------Instructor -large ------------------- # # # 잘안됨
# model_name = "hkunlp/instructor-large"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# embeddings = HuggingFaceInstructEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs,
#     embed_instruction="Represent the Korean patent writing passage for retrieval",#Represent chunks of Korean patent documents for similarity search.",
#     query_instruction="Represent the summary of Korean patent idea for retrieving relevant passages"
# )

# # ---------------- Multilingual e5 ----------------- # # # 잘됨 
model_name = "intfloat/multilingual-e5-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

## https://huggingface.co/intfloat/multilingual-e5-large
## this model requires prepending "query: " to texts if the texts are for semantic similarity search task
for doc in all_chunks:
    doc.page_content = "query: " + doc.page_content
print(all_chunks[20:40])

# # ------------------- Kobert ----------------- # # # 내가 쓰는 방법을 잘 몰라서 그런지 에러가 남 ㅠㅠ;;
# model_name = "skt/kobert-base-v1" # -> 실제로 embedding 할때 vectorstore = FAISS.from_documents(all_chunks, embedding=embeddings) 이거 할 때 에러가 난다.ㅠㅠ
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# embeddings = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs,
# )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Document(page_content='query: 상기 목적을 달성하기 위한 본 발명의 일 실시예에 의한 열전소자를 이용한 백라이트 유닛의 냉각 시스템은 백라이트 유닛과 연결되어 상기 백라이트 유닛에서 발생하는 열을 냉각하는 열전소자를 이용하는 백라이트 유닛의 냉각 시스템에 있어서, 상기 백라이트 유닛의 열원으로부터 열을 흡수하여 열전소자에 열을전달하는 내부 열흡수부; 그 일면은 상기 내부 열흡수부로부터 열을 흡수하여 냉각하고, 다른 면은 흡수한열을 방출하는 열전소자; 및 상기 열전소자로부터 방출된 열을 흡수하여 외부로 방출하는 외부 발열부를포함한다.또한, 상기 내부 열흡수부 및 상기 외부 발열부는 열전도성이 큰 물질로 이루어지는 것을 특징으로 하고,상기 열전소자는 상기 열전소자와 상기 내부 열 흡수부 또는 상기 열전소자와 상기 외부 발열부 사이에 방열판을 더 포함하는 것을 특징으로 한다.또한, 본 발명에서 백라이트 유닛은, LED 광원; 상기 LED광원을 구동하기 위한 구동 PCB; 상기 LED 광원으로부터의 광을 분산시켜 LCD 후면에 위치한 제1 광도광판으로 입력시키기 위한 제2 광도광판; 및 상기 제2광도광판에서 분산된 광이 수직으로 상기 제1 도광판에 입사되도록 하는 렌즈를 포함하는 것을 특징으로한다.한편, 본 발명의 다른 실시예에 의한 열전소자를 이용한 백라이트 유닛의 냉각 방법은 백라이트 유닛과 연결되어 상기 백라이트 유닛에서 발생하는 열을 냉각하는 열전소자를 이용한 백라이트 유닛의 냉각 방법에있어서, 상기 백라이트 유닛의 열원으로부터 열을 흡수하여 열전소자에 열을 전달하는 단계; 상기 열전소자가 일면에서 상기 내부 열흡수부로부터 방출된 열을 흡수하여 이를 냉각하고, 다른 면에서 흡수한 열을방출하는 단계; 및 상기 열전소자로부터 방출된 열을 흡수하여 외부로 방출하는 단계를 포함한다.이때, 상기 열전소자에 열을 전달하는 단계 및 상기 열전소자로부터 방출된 열을 흡수하는 단계는 전도성이 큰 물질에 의해 이루어지는 것을 특징으로 하고, 상기 열전소

In [8]:
# # -------------------Open ai ------------------ # # # Rate Limit 때문에 우선 한개씩 새로 add 하게 했음... 5개씩 add 해도 될듯!
# vectorstore = FAISS.from_documents([all_chunks[0]], embedding=embeddings)
# for chunk in tqdm(all_chunks[1:]):  # used for loop since if input all chunks at once, RATE LIMIT error occurred.
#     vectorstore.add_documents([chunk])
#     # time.sleep(2)

# # -------------------나머지  open source ------------------ # 
vectorstore = FAISS.from_documents(all_chunks, embedding=embeddings)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# retriever = vectorstore.as_retriever()
# retrieved_docs = retriever.invoke(
#     "냉각  제어  모듈,  이를  포함하는  손목  냉각  밴드  및  웨어러블  냉각  장치가  개시된다.  \
#     본  발명의  일  측면에따르면, 신체에 냉감을 일으키는 냉각 제어 모듈로서, 흡열표면과 발열표면을 가지는 열전소자와; \
#     상기 열전소자에 전원을 공급하기 위한 전원 공급부와; 상기 열전소자의 발열표면에 부착되어 열을 발산하는 방열부재(放熱部材)를  포함하며,\
#     상기  방열부재는,  일정  두께를  갖고  두께  방향으로  상기  열을  이동시키는  열분해  흑연  패드(pyrolytic graphite pad)와 \
#     금속 방열판이 교대로 적층되어 형성되는, 냉각 제어 모듈이 제공된다."
# )
# print(retrieved_docs)
# for doc in retrieved_docs:
#     print(doc.metadata['application_number'])
#     print(doc.page_content)

In [9]:
# 또다른 방법 :k 를 지정할 수 있다.
query = "냉각  제어  모듈,  이를  포함하는  손목  냉각  밴드  및  웨어러블  냉각  장치가  개시된다.  \
    본  발명의  일  측면에따르면, 신체에 냉감을 일으키는 냉각 제어 모듈로서, 흡열표면과 발열표면을 가지는 열전소자와; \
    상기 열전소자에 전원을 공급하기 위한 전원 공급부와; 상기 열전소자의 발열표면에 부착되어 열을 발산하는 방열부재(放熱部材)를  포함하며,\
    상기  방열부재는,  일정  두께를  갖고  두께  방향으로  상기  열을  이동시키는  열분해  흑연  패드(pyrolytic graphite pad)와 \
    금속 방열판이 교대로 적층되어 형성되는, 냉각 제어 모듈이 제공된다."
docs = vectorstore.similarity_search(query, k=10)
for doc in docs:
    print(doc.metadata['application_number'])
    print(doc.page_content)

1020177009557
query: 본 발명은 청구항 제1항의 전제부에 따른 손목 밴드 및 손목 밴드 상에 배치되는 냉각기를 구비한 휴대용 냉각장치에 관한 것이다.
1020177009557
query: 본 발명의 목적은 공지된 장치에 비해 전력 소비가 작으면서도 동일한 충전 시간에서 더 긴 작동 시간을 갖도록구성된,  손목 밴드 및 손목 밴드 상에 배치되는 냉각기를 구비한 휴대용 냉각 장치를 제공하는 것이다. 또한,본 발명에 따른 냉각 장치는 착용시 눈에 잘 띄지 않아야 하며, 또한 예컨대 탁상에서 손목 거치대가 구비된 컴퓨터로 작업시 적절한 의류와 함께 사용될 수도 있다.
1020177009557
query: 상기 목적은 본 발명의 청구항 제1항의 특징에 의해 달성된다. 본 발명에 따른, 손목 밴드 및 손목 밴드 상에배치되는 냉각기를 구비한 휴대용 냉각 장치에 있어서, 냉각기는 냉각기의 배출 방향으로 지향하는 기류의 생성을 위한 기류 발생기로서 디자인되고, 기류 발생기는 적어도 배출 방향으로 연장되는 연장부를 갖는 손목 밴드상의 장착부에 부착되는데, 이러한 연장부는 적어도 부분적으로 기류 내에 또는 기류에 위치하거나 적어도 부분적으로 기류를 둘러싸도록 구성된다. 본 발명에 따르면, 홀더 내에 배치되는 기류 발생기가 제공되며, 상기 기류의 배출 방향은 손목 밴드가 닫힌 상태,  즉 손목 밴드가 피부 표면 방향으로 착용된 상태에서 안쪽으로 향하도록 구성된다. 배출 방향으로 적절한공기 흐름을 지원하는 것은 냉각 장치의 기능에 필수적이며, 이는 본 발명에 따른 배출 방향으로 연장되는 연장부에 의해 구현되는데, 이러한 연장부는 적어도 부분적으로 기류 내에 또는 기류에 위치하거나 적어도 부분적으로 기류를 둘러싸도록 구성된다.  이러한 방식으로,  손목 밴드의 착용시 - 상세히 설명되는 바와 같이- 기류의냉각 또는 배출 영역이 제공됨에 따라, 냉각시킨 기류를 피부 표면에 분출시킴으로써 최대의 냉각 효과가 달성되도록  구성된다.  손목  밴드는  일반적인  방식으로  손

결과:


# Preprocessing

**logs**

23.11.15

- 1) problem with extracting reference (number) from texts -> some reference codes are not just numbers. it's number + alphabet e.g. 202a, 202b, or sometimes just uppercase alphabets e.g. T, SR~SZ
- 2) so I tried to first extract the reference codes from "부호에 대한 설명" column. (Because then I can find those specific reference codes inside of texts.) But extracting reference codes from "부호에 대한 설명" was difficult because currently all \<code\>: \<description\> string pairs are concatenated without \n and the parsing reference codes by just relying on regular expression was almost impossible. (some reference codes were uppercase alphabets, but including uppercase alphabets for codes caused problems.) So I asked Mooho if he could leave \n characters left for "부호에 대한 설명" section. He said yes, and I paused developing regex rules further.  

23.11.16
- I proceeded with some noise in reference extraction. 
- I also added some negative samples (3 random samples that are not prior arts of the source patent.)

In [ ]:
sample_data = pd.read_excel('data_preprocess/sample_data.xlsx',index_col=0)

## image to numbers, numbers to image
num2drawing_dicts = get_number2drawing_dict('/Users/hayley/Documents/p4ds/patent_search/data_preprocess/mock_data.json')

In [ ]:
# c.f. hupd에서 쓴 컬럼들 
#     "abstract": "...", # 요약 -> 우리나라 특허에는 abstract가 따로 있는거 같진 않고 요약 1개 섹션임
#     "claims": "...", # 청구범위
#     "background": "...", # 기술분야 + 배경기술
#     "summary": "...", # 요약? 
#     "full_description": "..." # 해결하려는과제 + 과제의해결수단 + 발명의효과 + 발명을실시하기위한구체적인내용???

In [ ]:
sample_data.columns 
# text embedding 대상이 되는 컬럼
# '요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
#        '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명'

# 각 섹션을 따로 따로 임베딩하는게 나을지 아니면은, 몇 섹션은 합치는게 나을지 고민이다. -> 일단 빠르게 ㄱㄱ!

In [ ]:
text_columns = ['요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
       '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명']

In [ ]:
# # c.f. 컬럼별 길이 분포 
# all_data = pd.read_excel('/Users/hayley/Documents/p4ds/patent_search/pdf_process/data_large.xlsx', index_col=0)
# for col in all_data.columns:
#     try:
#         if "extracted_numbers" not in col:
#             lens = all_data[col].apply(lambda x: len(str(x)))
#             print(col, lens.mean().round(2))
#     except:
#         continue
# # id 13.04
# # 요약 301.69
# # 대표도 395.03
# # 청구범위 3565.65
# # 기술분야 150.39
# # 배경기술 1713.51
# # 해결하려는과제 210.42
# # 과제의해결수단 1653.4
# # 발명의효과 209.81
# # 도면의간단한설명 1206.95
# # 발명을실시하기위한구체적인내용 17958.91
# # 부호의설명 188.81

In [ ]:
patent_chunk_dicts=[]

for i, row in sample_data.iterrows():
    patent_dict = dict( 
        # 특허 번호 따기
        application_number = str(row.id), # 출원 번호
        publication_number = '', # 공개 번호
        patent_number = '', # 등록 번호
        chunks = [],
        # chunks_wo_drawing_and_numbers_desc = []
    )
    print(f"patent {row.id} 's chunk sizes")
    
    # text column들을 돌면서, chunking 하기
    chunks = []
    drawing_nums_list = []
    refs_list = []
    # chunks_wo_drawing_desc = [] # 도면의 간단한 설명, 부호의 설명 제외.. false positive 가 생기지는 않을까?
    for col in text_columns:
        if (col in ['도면의간단한설명', '부호의설명']) or (str(row[col]) == 'nan'):
            continue
        curr_section_chunks = text_splitter.create_documents([str(row[col])], [{"application_number": str(row.id)}])
        chunks.extend(curr_section_chunks)
        
        # chunks_wo_drawing_desc.extend(curr_section_chunks)
        print(col, len(curr_section_chunks), end=' | ')
        
        # chunk 별로 reference 발생한 부호 찾기
        for chnk in curr_section_chunks:
            refs = extract_ref_from_text(chnk)
            refs_list.append(refs)
            
            #  부호가 있었으면 drawing number 로 한번 또 치환하기
            if len(refs) > 0: # if not empty, convert found references to drawing numbers
                drawing_nums = convert_refs_to_drawing_num(refs, num2drawing_dicts[str(row.id)]['num2drawing'])
            else:
                drawing_nums = []
            drawing_nums_list.append(drawing_nums)
                
        
    # patent_dict['chunks'] = list(zip(chunks, zip(refs_list, drawing_nums_list)))
    patent_dict['chunks'] = chunks
    # patent_dict['chunks_wo_drawing_and_numbers_desc'] = chunks_wo_drawing_desc
    print()
    patent_chunk_dicts.append(patent_dict)    
    

In [ ]:
json.dump(patent_chunk_dicts, open('data_preprocess/sample_chunk_data.json', 'w'), ensure_ascii = False )

In [ ]:
# for col in select_columns:
#     sample_data[f"{col}_ref"] = sample_data[col].apply(extract_ref_from_text)

# sample_data['도면의간단한설명_dict'] = sample_data['도면의간단한설명'].apply(extract_description_for_image)

# sample_data.loc[:,'부호의설명_dict'] = sample_data['부호의설명'].apply(extract_description_for_code)

# for col in select_columns:
#     sample_data[f"{col}_ref_drawing"] = sample_data.apply(convert_ref_to_drawing_num, args=(col,image_df), axis=1)

# OCR

- Tesseract : 잘 못함
- 다른 오픈소스 : pyocr -> tesseract랑 똑같음 / calamari-ocr -> tensorflow 설치해야 되고 등등 maintain이 잘 안되는 패키지 인듯
- GCP vision api : 일단 ui 에서 테스트 해보고 copy json output 버튼 눌러가지고 json output 복사해와서 json output에서 텍스트만 파싱하는 것만 짬

In [ ]:
import pytesseract
from PIL import Image

## tesseract를 먼저 다운받고, 다운받은 경로를 넣어주어야 함.
## You need to first download tesseract & insert the path to the exe file below.
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.3.3/bin/tesseract'

In [ ]:
# text image to string
from glob import glob

image_paths = glob('data_preprocess/image/*/*.png')

for img in image_paths:
    print(img)
    print(pytesseract.image_to_string(Image.open(img), lang='eng'))

In [ ]:
for i in json_output['fullTextAnnotation']['pages'][0]['blocks']:
    for j in i['paragraphs']:
        for k in j['words']:
            curr_word = ''
            for l in k['symbols']:
                curr_word += l['text']
            if curr_word.startswith('-'):
                curr_word = curr_word[1:]
            print(curr_word)

In [ ]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

In [ ]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)